In [43]:
%%capture
!pip install -r requirement.txt;

In [44]:
import numpy as np
import geatpy as ea
import pandas as pd
import random
from datetime import datetime
print(ea.__version__)
pd.set_option('display.max_columns', 500) # show all columns
INTINF = 1000

debug = 1 # debug mode

# randomly choose n samples
n = 30000

2.7.0


### PARAMETER INPUT

In [45]:
# boolen
NbvCost = False # True: max nbv; False: min cost

# float
minTotalNbv = 10000000
maxTotalNbv = None

# float
minTotalCost = None
maxTotalCost = 100000000

# float
fleetAgeLowBound = [3, None, None]
fleetAgeUpBound = [INTINF, None, None]
fleetAgeLimit = [0.1, None, None]

# float
OnHireLimit = 0.1

# float
weightedAgeLowBound = [-INTINF, 4, None]
weightedAgeUpBound = [3.3, 8, None]
weightedAgeLimit = [0.1, 0.1, None]

# string
productType = ['D40', 'D20', None]
# float
productLimit = [0.1, 0.1, None]

# string
lesseeType = ['MSC', 'ONE', 'HAPAG']
# float
lesseeLimit = [0.1, 0.1, 0.1]

# string
contractType = ['LE', 'LF', 'LT']
# float
contractLimit = [0.1, 0.1, 0.1]

### READ DATA
TODO: read data from GP

In [46]:
rawData = pd.read_excel(io='./FCI ANZ (2022-07-08) (NBV as at 30 Jun 2022)_v2.xlsx', sheet_name='Raw (portfolio)', engine='openpyxl')
rawData = rawData.iloc[:30000, :61]

In [47]:
data = rawData.sample(n)
print(data.shape)
data.sample(3)

(30000, 61)


,Business Unit (Contract),Business Unit Am,Unit Id Fz,Cost,Po Num Fz,Mfr Dt Fz,Asset Id,Pool Fz,Product,Birth Unit Id Fz,Owner,Teu Fz,Ceu Fz,Contract Cust Id,Contract Num,Contract Dt,Contract Expiration Dt,Contract Lease Type,Contract Currency,Exchange Rate,Per Diem Rate (Max),Per Diem Rate (Max) in USD,Contract Rate Type (Min),Per Diem Rate (Min),Per Diem Rate (Min) in USD,Buy Out Amt Fz,Monthly Depr,Depre adjustment,Nbv,Life Status Fz,Billing Status Fz,Move Status Fz,Move Type Fz,Inv Status Fz,Fleet Year Fz,Financing Cd,Asset Status,Lessor,Remaining Lease Term,Age x CEU,RML x CEU,Mfr Year,Reserve Code Fz,Remaining Lease Term(C),RML x CEU(C),Chk on RML validity(HAPAG),Expiry year,Expiry month,Chk on RML validity(12 YR),Planned,Entity,Correct CEU,NP year,NBV check,age at drawdown X CEU,RML at drawdown X CEU,category,string,filter 2017 contracts,contract units,filter no. of units
6126,FZOPS,DF3RM,FDCU0088898,3682.0,DFIC-FL-1801,1/1/2018,1096073.0,DFC3,D4H,FDCU0088898,Dong Fang,2.0,1.7,MSC,LT-MSC-35B,2018-03-01,2032-07-31,LT,USD,1.0,1.07,1.07,ST,0.39,0.39,0.0,-15.42,0.674,2933.00,AC,ON,GO,RLI,AVA,4.49,NaN,I,FUSBI,10.095890,7.633,17.163014,2018.0,F - OL / < 8 yrs,10.095890,17.163014,0.0,2032.0,7.0,14.585890,FCI ANZ 100M,FCI,1.7,2018 NP,Reasonable,8.909164,15.886849,from ANZ portfolio,LT-MSC-35BD4H,NaN,5621.0,1.0
18462,FZOPS,DF3RM,FFAU4052854,6000.0,CSIH-FL-2115,4/12/2021,1708182.0,DFC3,D4H,FFAU4052854,Dong Fang,2.0,1.7,MSC,LT-MSC-48,2021-04-01,2029-06-30,LT,USD,1.0,1.95,1.95,FR,1.95,1.95,720.0,-27.69,0.000,5612.35,AC,ON,GO,RLI,AVA,1.22,NaN,I,FUSBI,7.008219,2.074,11.913973,2021.0,F - FL / < 8 yrs,7.008219,11.913973,0.0,2029.0,6.0,8.228219,FCI ANZ 100M,FCI,1.7,2021 NP,Reasonable,3.350164,10.637808,from ANZ portfolio,LT-MSC-48D4H,NaN,6997.0,1.0
14092,FZOPS,DF3RM,FFAU4065450,6007.0,SMSL-FL-2105,4/28/2021,1721942.0,DFC3,D4H,FFAU4065450,Dong Fang,2.0,1.7,MSC,LT-MSC-48,2021-04-01,2029-06-30,LT,USD,1.0,1.95,1.95,FR,1.95,1.95,720.0,-27.73,0.000,5618.80,AC,ON,GO,RLI,AVA,1.17,NaN,I,FUSBI,7.008219,1.989,11.913973,2021.0,F - FL / < 8 yrs,7.008219,11.913973,0.0,2029.0,6.0,8.178219,FCI ANZ 100M,FCI,1.7,2021 NP,Reasonable,3.265164,10.637808,from ANZ portfolio,LT-MSC-48D4H,NaN,6997.0,1.0


### DATA PREPARATION

#### Container Age

Assign to `FleetAge1`, `FleetAge2`, `FleetAge3`

In [48]:
def SelectFleetAge(age, i):
    # check input
    if fleetAgeLowBound[i] is None:
        fleetAgeLowBound[i] = -float('inf')
    if fleetAgeUpBound[i] is None:
        fleetAgeUpBound[i] = float('inf')
        
    if fleetAgeLowBound[i] <= age < fleetAgeUpBound[i]:
        age = 1
    else:
        age = 0
    return age

data['FleetAge1'] = data.apply(lambda x: SelectFleetAge(x['Fleet Year Fz'], 0), axis=1)
data['FleetAge2'] = data.apply(lambda x: SelectFleetAge(x['Fleet Year Fz'], 1), axis=1)
data['FleetAge3'] = data.apply(lambda x: SelectFleetAge(x['Fleet Year Fz'], 2), axis=1)

#### Status

Assign new column `Status`

In [49]:
def SelectStatus(status):
    if status == 'ON':
        status = 1
    else:
        status = 0
    return status

data['Status'] = data.apply(lambda x: SelectStatus(x['Billing Status Fz']), axis=1)

#### Weighted Age

Assign new column `WeightedAge1`, `WeightedAge2`, `WeightedAge3`

In [50]:
def SelectWeightedAge(age, i):
    # check input
    if weightedAgeLowBound[i] is None:
        weightedAgeLowBound[i] = -float('inf')
    if weightedAgeUpBound[i] is None:
        weightedAgeUpBound[i] = float('inf')
        
    if weightedAgeLowBound[i] <= age < weightedAgeUpBound[i]:
        age = 1
    else:
        age = 0
    return age

data['WeightedAge1'] = data.apply(lambda x: SelectWeightedAge(x['Age x CEU'], 0), axis=1)
data['WeightedAge2'] = data.apply(lambda x: SelectWeightedAge(x['Age x CEU'], 1), axis=1)
data['WeightedAge3'] = data.apply(lambda x: SelectWeightedAge(x['Age x CEU'], 2), axis=1)

#### Product

Assign new column `ProductType1`, `ProductType2`, `ProductType3`

In [51]:
def SelectProductType(product, i):
    if product == productType[i]:
        product = 1
    else:
        product = 0
    return product

data['ProductType1'] = data.apply(lambda x: SelectProductType(x['Product'], 0), axis=1)
data['ProductType2'] = data.apply(lambda x: SelectProductType(x['Product'], 1), axis=1)
data['ProductType3'] = data.apply(lambda x: SelectProductType(x['Product'], 2), axis=1)

#### Lessee

Assign new column `Lessee1`, `Lessee2`, `Lessee3`

In [52]:
def SelectLessee(lessee, i):
    if lessee == lesseeType[i]:
        lessee = 1
    else:
        lessee = 0
    return lessee

data['Lessee1'] = data.apply(lambda x: SelectLessee(x['Contract Cust Id'], 0), axis=1)
data['Lessee2'] = data.apply(lambda x: SelectLessee(x['Contract Cust Id'], 1), axis=1)
data['Lessee3'] = data.apply(lambda x: SelectLessee(x['Contract Cust Id'], 2), axis=1)

#### Contract Type

Assign new column `ContractType1`, `ContractType2`, `ContractType3`

In [53]:
def SelectContractType(contract, i):
    if contract == contractType[i]:
        contract = 1
    else:
        contract = 0
    return contract

data['ContractType1'] = data.apply(lambda x: SelectContractType(x['Contract Lease Type'], 0), axis=1)
data['ContractType2'] = data.apply(lambda x: SelectContractType(x['Contract Lease Type'], 1), axis=1)
data['ContractType3'] = data.apply(lambda x: SelectContractType(x['Contract Lease Type'], 2), axis=1)

### SAVE DATA

In [54]:
if debug:
    data.to_csv('prepared_data_demo.csv')

In [55]:
if debug:
    data = pd.read_csv('./prepared_data_demo.csv')
data.sample(3)

,Unnamed: 0,Business Unit (Contract),Business Unit Am,Unit Id Fz,Cost,Po Num Fz,Mfr Dt Fz,Asset Id,Pool Fz,Product,Birth Unit Id Fz,Owner,Teu Fz,Ceu Fz,Contract Cust Id,Contract Num,Contract Dt,Contract Expiration Dt,Contract Lease Type,Contract Currency,Exchange Rate,Per Diem Rate (Max),Per Diem Rate (Max) in USD,Contract Rate Type (Min),Per Diem Rate (Min),Per Diem Rate (Min) in USD,Buy Out Amt Fz,Monthly Depr,Depre adjustment,Nbv,Life Status Fz,Billing Status Fz,Move Status Fz,Move Type Fz,Inv Status Fz,Fleet Year Fz,Financing Cd,Asset Status,Lessor,Remaining Lease Term,Age x CEU,RML x CEU,Mfr Year,Reserve Code Fz,Remaining Lease Term(C),RML x CEU(C),Chk on RML validity(HAPAG),Expiry year,Expiry month,Chk on RML validity(12 YR),Planned,Entity,Correct CEU,NP year,NBV check,age at drawdown X CEU,RML at drawdown X CEU,category,string,filter 2017 contracts,contract units,filter no. of units,FleetAge1,FleetAge2,FleetAge3,Status,WeightedAge1,WeightedAge2,WeightedAge3,ProductType1,ProductType2,ProductType3,Lessee1,Lessee2,Lessee3,ContractType1,ContractType2,ContractType3
23165,8242,FZOPS,DF3RM,FCIU7425202,3702.0,SMSL-FL-1806,6/11/2018,1243868.0,DFC3,D4H,FCIU7425202,Dong Fang,2.0,1.7,MSC,LT-MSC-35B,2018-03-01,2032-07-31,LT,USD,1.0,1.07,1.07,ST,0.39,0.39,0.0,-15.57,0.670,2967.91,AC,ON,GO,RLI,AVA,4.05,NaN,I,FUSBI,10.09589,6.885,17.163014,2018.0,F - OL / < 8 yrs,10.09589,17.163014,0.00000,2032.0,7.0,14.14589,FCI ANZ 100M,FCI,1.7,2018 NP,Reasonable,8.161164,15.886849,from ANZ portfolio,LT-MSC-35BD4H,NaN,5621.0,1.0,1,1,1,1,0,1,1,0,0,0,1,0,0,0,0,1
11466,29943,FZOPS,DF3RM,FBIU0102119,2182.0,DFIC-FL-1801,1/2/2018,1080395.0,DFC3,D20,FBIU0102119,Dong Fang,1.0,1.0,HAPAG,LF-HAPAG-09,2017-11-01,2030-01-31,LF,USD,1.0,0.65,0.65,FR,0.35,0.35,0.0,-7.76,0.629,1784.49,AC,ON,GO,RLI,AVA,4.49,NaN,I,FUSBI,7.59726,4.490,7.597260,2018.0,F - OL / < 8 yrs,7.51000,7.510000,-0.08726,2030.0,1.0,12.00000,CNIC,FCI,1.0,2018 NP,Reasonable,5.240685,6.759315,from available portfolio,LF-HAPAG-09D20,NaN,1494.0,1.0,1,1,1,1,0,1,1,0,1,0,0,0,1,0,1,0
19592,24220,FZOPS,DF3RM,FBIU0328472,2267.0,CIMC-FL-1804,3/22/2018,1183193.0,DFC3,D20,FBIU0328472,Dong Fang,1.0,1.0,ONE,LE-ONE-01,2018-02-01,2026-10-14,LE,USD,1.0,0.69,0.69,FA,0.69,0.69,0.0,-8.26,0.607,1896.17,AC,ON,GO,RLI,AVA,4.27,NaN,I,FUSBI,4.29589,4.270,4.295890,2018.0,F - OL / < 8 yrs,4.29589,4.295890,0.00000,2026.0,10.0,8.56589,CNIC,FCI,1.0,2018 NP,Reasonable,5.020685,3.545205,from available portfolio,LE-ONE-01D20,NaN,4991.0,1.0,1,1,1,1,0,1,1,0,1,0,0,1,0,1,0,0


### Model Part

In [56]:
nbv = data['Nbv'].to_numpy().reshape(n, 1)
cost = data['Cost'].to_numpy().reshape(n, 1)
fleetAge = np.stack([data['FleetAge1'].to_numpy(),
                     data['FleetAge2'].to_numpy(),
                     data['FleetAge3'].to_numpy()]).reshape(3, n, 1)
status = data['Status'].to_numpy().reshape(n, 1)
weightedAge = np.stack([data['WeightedAge1'].to_numpy(),
                        data['WeightedAge2'].to_numpy(),
                        data['WeightedAge3'].to_numpy()]).reshape(3, n, 1)
productType = np.stack([data['ProductType1'].to_numpy(),
                        data['ProductType2'].to_numpy(),
                        data['ProductType3'].to_numpy()]).reshape(3, n, 1)
lessee = np.stack([data['Lessee1'].to_numpy(),
                   data['Lessee2'].to_numpy(),
                   data['Lessee3'].to_numpy()]).reshape(3, n, 1)
contractType = np.stack([data['ContractType1'].to_numpy(),
                         data['ContractType2'].to_numpy(),
                         data['ContractType3'].to_numpy(),]).reshape(3, n, 1)
if debug:
    print('nbv shape: ', nbv.shape)
    print('cost shape:', cost.shape)
    print('fleetAge shape: ', fleetAge.shape)
    print('status shape:', status.shape)
    print('weightedAge shape:', weightedAge.shape)
    print('productType shape:', productType.shape)
    print('lessee shape:', lessee.shape)
    print('contractType shape:', contractType.shape)

nbv shape:  (30000, 1)
cost shape: (30000, 1)
fleetAge shape:  (3, 30000, 1)
status shape: (30000, 1)
weightedAge shape: (3, 30000, 1)
productType shape: (3, 30000, 1)
lessee shape: (3, 30000, 1)
contractType shape: (3, 30000, 1)


#### Constraints

`x`: np.array, size=num_of_container. `x[i]=1` stands for $i^{th}$ container is selected; `x[i]=0` stands for $i^{th}$ container is not selected.

1. minTotalNbv <= Total NBV <= maxTotalNbv
    
2. minTotalCost <= Total Cost <= maxTotalCost
    
3. Container Age
    
4. Status:

    1. OnHire >= OnHireLimit

5. Weighted Age

6. Product
    
7. Lessee
    
8. Contract Type


#### Objective

1. min Cost: `Vars @ cost`

2. max Nbv: `Vars @ nbv`

#### Define Problem
Define objective function and constraint functions

In [63]:
def DEMO():
    class MyProblem(ea.Problem):
        def __init__(self):
            name ='MyProblem'
            M = 1
            maxormins = [-1 if NbvCost else 1] # -1: max, 1: min
            Dim = n # num of variables
            varTypes = [1] * Dim # integers: 0 or 1
            lb = [0] * Dim # low bound
            ub = [1] * Dim # up bound
            lbin = [1] * Dim # low bound include
            ubin = [1] * Dim # up bound include
            ea.Problem.__init__(self, name, M, maxormins, Dim, varTypes, lb,ub, lbin, ubin)
            
        def aimFunc(self, pop):
            Vars = pop.Phen
            
            # objective function
            pop.ObjV = Vars @ nbv if NbvCost else Vars @ cost

            numSelectedContainer = Vars.sum(axis=1, keepdims=True)
            CVList = []
            # NBV
            if minTotalNbv is not None:
                CVList.append(-(Vars @ nbv - minTotalNbv))
            if maxTotalNbv is not None:
                CVList.append(Vars @ nbv - maxTotalNbv)
            # Cost
            if minTotalCost is not None:
                CVList.append(-(Vars @ cost - minTotalCost))
            if maxTotalCost is not None:
                CVList.append(Vars @ cost - maxTotalCost)
            # Container age
            if fleetAgeLimit[0] is not None:
                CVList.append(-((Vars @ fleetAge[0]) / numSelectedContainer - fleetAgeLimit[0]))
            if fleetAgeLimit[1] is not None:
                CVList.append(-((Vars @ fleetAge[1]) / numSelectedContainer - fleetAgeLimit[1]))
            if fleetAgeLimit[2] is not None:
                CVList.append(-((Vars @ fleetAge[2]) / numSelectedContainer - fleetAgeLimit[2]))
            # Status
            if OnHireLimit is not None:
                CVList.append(-((Vars @ status) / numSelectedContainer - OnHireLimit))
            # Weighted age
            if weightedAgeLimit[0] is not None:
                CVList.append(-((Vars @ weightedAge[0]) / numSelectedContainer - weightedAgeLimit[0]))
            if weightedAgeLimit[1] is not None:
                CVList.append(-((Vars @ weightedAge[1]) / numSelectedContainer - weightedAgeLimit[1]))
            if weightedAgeLimit[2] is not None:
                CVList.append(-((Vars @ weightedAge[2]) / numSelectedContainer - weightedAgeLimit[2]))
            # Product
            if productLimit[0] is not None:
                CVList.append(-((Vars @ productType[0]) / numSelectedContainer - productLimit[0]))
            if productLimit[1] is not None:
                CVList.append(-((Vars @ productType[1]) / numSelectedContainer - productLimit[1]))
            if productLimit[2] is not None:
                CVList.append(-((Vars @ productType[2]) / numSelectedContainer - productLimit[2]))
#             # Lessee
#             if lesseeLimit[0] is not None:
#                 CVList.append(-((Vars @ lessee[0]) / numSelectedContainer - lesseeLimit[0]))
#             if lesseeLimit[1] is not None:
#                 CVList.append(-((Vars @ lessee[1]) / numSelectedContainer - lesseeLimit[1]))
#             if lesseeLimit[2] is not None:
#                 CVList.append(-((Vars @ lessee[2]) / numSelectedContainer - lesseeLimit[2]))
#             # Contract type
#             if contractLimit[0] is not None:
#                 CVList.append(-((Vars @ contractType[0]) / numSelectedContainer - contractLimit[0]))
#             if contractLimit[1] is not None:
#                 CVList.append(-((Vars @ contractType[1]) / numSelectedContainer - contractLimit[1]))
#             if contractLimit[2] is not None:
#                 CVList.append(-((Vars @ contractType[2]) / numSelectedContainer - contractLimit[2]))
                
            # constraints functions
            pop.CV = np.hstack(CVList)


    problem = MyProblem()
    Encoding ='RI'
    NIND = 100 # init number of population instance
    Field = ea.crtfld(Encoding, problem.varTypes, problem.ranges,problem.borders)
    population = ea.Population(Encoding, Field, NIND)

    # set parameters
    myAlgorithm = ea.soea_DE_best_1_L_templet(problem, population)
    myAlgorithm.MAXGEN = 10000
    myAlgorithm.mutOper.F = 0.5
    myAlgorithm.recOper.XOVR = 0.7 # probability of chrom crossover
    myAlgorithm.logTras = 1 # record log each logTras generations
    myAlgorithm.trappedValue = 1e-6
    myAlgorithm.maxTrappedCount = 100 # TODO: increase it or remove it

    res = ea.optimize(myAlgorithm, seed=2049, verbose=False, drawing=1, outputMsg=True, drawLog=False, saveFlag=False, dirName='result')
    if debug:
        print(res)
    return res

In [ ]:
selectedPackage = DEMO()

#### Output

In [15]:
print('Time Consumption: ', np.round(selectedPackage['executeTime'], decimals=2), 's')
print(selectedPackage['stopMsg'])

if selectedPackage['success']:
    print('Algorithm Succeeded!')
    print('===================================================================================================')
    print('Optimal Target Value: ', np.round(selectedPackage['ObjV'].item(), decimals=2))
    print('Constraint Violation: ', np.round(selectedPackage['CV'][0], decimals=2))
    if debug:
        # TODO: doing
        print('===================================================================================================')
        Vars = selectedPackage['Vars'][0]
else:
    print('No Feasible Results...')

Time Consumption:  9.33 s
The algotirhm stepped because it exceeded the trapped count limit.
Algorithm Succeeded!
Optimal Target Value:  3478773.49
Constraint Violation:  [-2.47877349e+06 -9.65212265e+07 -3.01416240e+06 -9.59858376e+07
 -6.30000000e-01 -0.00000000e+00 -0.00000000e+00 -1.00000000e+00
 -0.00000000e+00 -2.00000000e-01 -0.00000000e+00 -0.00000000e+00
 -2.00000000e-01 -0.00000000e+00 -4.30000000e-01 -3.60000000e-01
 -1.90000000e-01 -2.50000000e-01 -1.90000000e-01 -5.70000000e-01]


In [16]:
# print final result
print('Selected Package:')
print('===================================================================================================')
print(len(selectedPackage['Vars'].nonzero()[1]), '/', n, 'containers selected.')
print(selectedPackage['Vars'].nonzero()[1]) # index begins from 0
print('===================================================================================================')
print(selectedPackage['Vars'][0].tolist())

Selected Package:
988 / 1000 containers selected.
[  0   1   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 105 106 107 108 109
 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127
 128 129 130 131 132 133 134 135 136 137 138 139 140 142 143 144 145 146
 147 148 149 150 151 152 153 154 155 157 158 159 160 161 162 163 164 165
 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183
 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201
 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219
 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237
 